In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg 

import pandas as pd
root = '../dataset/hi'
root_soh = '../dataset/soh_charge'


In [2]:

def load_and_process(root, root_soh, dataset):
    hiv = pd.read_csv(f'{root}/hiv-charge-CS2_{dataset}.csv')
    soh = pd.read_csv(f'{root_soh}/soh-CS2_{dataset}.csv')

    # Mantieni solo le colonne da hiv e soh
    df = pd.merge(hiv, soh, how="right", on="cycle")[["cycle", "hi_v", "SOH"]]
    df = df.drop(columns=["cycle"])
    df_hiv = df[["hi_v"]]
    df_soh = df[["SOH"]]
    return df_hiv, df_soh

# Carica e processa i dataset
df_35_hiv, df_35_soh = load_and_process(root, root_soh, 35)
df_36_hiv, df_36_soh = load_and_process(root, root_soh, 36)
df_38_hiv, df_38_soh = load_and_process(root, root_soh, 38)


In [3]:
# Create sequences for training set
X_train = pd.concat([df_36_hiv, df_38_hiv]).values
y_train = pd.concat([df_36_soh, df_38_soh]).values

# Calculate mean and std for normalization
mean = X_train.mean()
std = X_train.std()

# Standardize the data
X_train = (X_train - mean) / std

# # Create sequences for testing set
X_test = df_35_hiv.values
y_test = df_35_soh.values

# # Normalize the test data using the mean and std from training data
X_test = (X_test - mean) / std

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(2029, 1) (2029, 1) (915, 1) (915, 1)


In [4]:
y_train.shape

(2029, 1)

In [5]:
# load the data
trainLen = len(X_train)
# testLen = len(X_test)
initLen = 100
print(trainLen, initLen)

2029 100


In [6]:
inSize = outSize = 1
resSize = 1000
a = 0.3 # leaking rate
np.random.seed(42)
Win = (np.random.rand(resSize,1+inSize) - 0.5) * 1
W = np.random.rand(resSize,resSize) - 0.5 
rhoW = max(abs(linalg.eig(W)[0]))
W *= 1.25 / rhoW

In [7]:
print(Win.shape)

(1000, 2)


In [8]:
# allocated memory for the design (collected states) matrix
# X = np.zeros((1+inSize+resSize,trainLen-initLen))
X = np.zeros((1+inSize+resSize, len(X_train)-initLen))
# set the corresponding target matrix directly
# Yt = data[None,initLen+1:trainLen+1] 
Yt = y_train[None,initLen:trainLen+1].reshape(1, -1)
# run the reservoir with the data and collect X
x = np.zeros((resSize,1))

print(Yt.shape, X.shape, x.shape)

# for t in range(trainLen):
#     u = data[t]
#     x = (1-a)*x + a*np.tanh( np.dot( Win, np.vstack((1,u)) ) + np.dot( W, x ) )
#     if t >= initLen:
#         X[:,t-initLen] = np.vstack((1,u,x))[:,0]

for t in range(trainLen):
    u = X_train[t]
    # print(t,np.vstack((1, u)).shape)
    # x = (1-a)*x + a*np.tanh(np.dot(Win, np.vstack((1, u))) + np.dot(W, x))
    x = (1-a)*x + a*np.tanh(np.dot(Win, np.vstack((1, u))) + np.dot(W, x))

    # y = np.dot(Wout, np.vstack((1, u, x)))
    # Y_train_pred[:, t] = y
    # u = y
    if t >= initLen:
        X[:,t-initLen] = np.vstack((1,u,x))[:,0]

(1, 1929) (1002, 1929) (1000, 1)


In [9]:
# train the output by ridge regression
reg = 1e-8  # regularization coefficient

# using scipy.linalg.solve:
Wout = linalg.solve( np.dot(X,X.T) + reg*np.eye(1+inSize+resSize), 
    np.dot(X,Yt.T) ).T


# run the trained ESN in a generative mode. no need to initialize here, 
# because x is initialized with training data and we continue from there.
Y = np.zeros((outSize,len(resSize)))
print(X_train.shape)
u = X_train[-1]

print(Y.shape, u.shape)

ValueError: array must not contain infs or NaNs

In [ ]:
for t in range(len(X_test)):
    x = (1-a)*x + a*np.tanh( np.dot( Win, np.vstack((1,u)) ) + np.dot( W, x ) )
    y = np.dot( Wout, np.vstack((1,u,x)) )
    Y[:,t] = y
    # generative mode:
    u = y
    ## this would be a predictive mode:
    #u = data[trainLen+t+1] 

In [ ]:

# compute MSE for the first errorLen time steps
errorLen = 500
mse = sum( np.square( y_test[trainLen+1:trainLen+errorLen+1] - 
    Y[0,0:errorLen] ) ) / errorLen
print('MSE = ' + str( mse ))
    

In [ ]:
# Plot dei risultati
plt.figure(1).clear()
plt.plot(y_test[:errorLen], 'g')
plt.plot(Y.T[:errorLen], 'b')
plt.title('Target and predicted signals')
plt.legend(['Target signal', 'Predicted signal'])
plt.show()

In [ ]:

# plot some signals
plt.figure(1).clear()
plt.plot( data[trainLen+1:trainLen+testLen+1], 'g' )
plt.plot( Y.T, 'b' )
plt.title('Target and generated signals $y(n)$ starting at $n=0$')
plt.legend(['Target signal', 'Free-running predicted signal'])

plt.figure(2).clear()
plt.plot( X[0:20,0:200].T )
plt.title(r'Some reservoir activations $\mathbf{x}(n)$')

plt.figure(3).clear()
plt.bar( np.arange(1+inSize+resSize), Wout[0].T )
plt.title(r'Output weights $\mathbf{W}^{out}$')

plt.show()